<h1><a href="http://papers.nips.cc/paper/6105-guided-policy-search-via-approximate-mirror-descent.pdf">Guided Policy Search via Approximate Mirror Descent</a></h1>


<h2>1. Summary</h2>
<li>Guided policy search algorithms can be interpreted as an approximate variant of mirror descent.</li>
<li>Propose a new guided policy search algorithm that is simpler and conditionally guarantees convergence.</li>
<li>AI safety problems are categorized into ``robustness`` and ``specification`` problems.</li>

<h2>2. Introduction</h2>

In generic guided policy search algorithm, performance gurantee only holds at convergence, because learnt policy and guiding trajectories are gradually matched.

Guided policy search via approximate mirror descent alternates between trajectory-centric reinforcement learning and supervised learning with a bound on policy cost controlling the step size.

<script>
%%javascript
MathJax.Hub.Register.StartupHook("TeX AMSmath Ready", function () {
  var AMS = MathJax.Extension['TeX/AMSmath'];
  MathJax.InputJax.TeX.postfilterHooks.Add(function (data) {
    var jax = data.script.MathJax;
    jax.startNumber = AMS.startNumber;
    jax.eqLabels = AMS.eqlabels;
    jax.eqIDs = AMS.eqIDs;
  });
  MathJax.InputJax.TeX.prefilterHooks.Add(function (data) {
    var jax = data.script.MathJax;
    if (jax.startNumber != undefined) {
      AMS.startNumber = jax.startNumber;
      Object.keys(jax.eqLabels).forEach(function (x) {delete AMS.labels[x]});
      Object.keys(jax.eqIDs).forEach(function (x) {delete AMS.IDs[x]});
    }
  }, 1);
});
</script>

<h2>3. Guided Policy Search</h2>

* **Classical policy search** algorithms solves optimization problem 

\begin{equation}
min \sum^N_{i=1} \mathbb{E}_{\pi_\theta{(\tau_i)}}[l(\tau_i)]
\end{equation}

where $l(\tau_i)$ is the loss of trajectory $\tau_i=\{x_1, u_1, ..., u_T, u_T\}$ and $\pi_\theta{(\tau_i)}=p(x_1|\tau_i)\prod^T_{t=1}p(x_{t+1}|x_t, u_t,\tau_i)\pi_\theta(u_t|x_t, \tau_i)$ is the policy. The optimization is based on samples drawn from running policies in real physical world. These kinds of methods are typically limited to policies with about 100 parameters.

* **Generic guided policy search** algorithm splits the optimization into two phases:
<ul>
    <li>**``Control phase``(C-step)**: Finds local optimal policies $p_i(u_t|x_t)$ by minimizing loss. Use newly solved $p_i(u_t|x_t)$ to generate new samples.</li>
    <li>**``Supervised phase``(S-step)**: Optimizes a global policy $\pi_\theta(u_t|x_t)$ to match the newly learnt local policies in all the generated samples.</li>
</ul>

* In ``C-step``, local policies are encouraged to stay close to the global policy learnt from last **S-step** as well as the previously learnt local policies. The optimization problem for local policies becomes

\begin{equation}
min_p \sum_\tau \mathbb{E}_{\tau\sim p} [l(\tau)+D_{KL}(p(u_t|x_t)||\pi_{\theta_{i-1}})]\qquad s.t. D_{KL}(p(\tau)||p_{i-1}(\tau))\leq \epsilon
\end{equation}

where $\pi_{\theta_{i-1}}$ and $p_{i-1}$ are global policy and local policies learnt from previous iteration.


* At convergence, it is enforced that $p(u_t|x_t)=\pi_\theta(u_t|x_t)$. But in the intermediate iterations, there is no constraint on the global policy $\pi_\theta(u_t|x_t)$. And practical application may preclude running the method to full convergence.

3. Mirror Descent Guided Policy Search



<h2>4. Mirror Descent Guided Policy Search</h2>

<li>Instead of constraining local policy against previously learnt local policy, it is now constraint against global policy.</li>
<li>Penalty of KL-divergence in the surrogate loss function is removed.</li>
<li>Assuming linear dynamics, quadratic cost function and convex policy space $\Pi_\Theta$, this method corresponds to an instance of **mirror descent**.

*Algorithm 2*
<ol>
    *<li>for iteration k in 1:K do</li>*
    *<li>$\qquad$ ``C-step``: $p_i=argmin_{p_i}\mathbb{E}_{p_i(\tau)}[\sum^T_{t=1}l(x_t, u_t)]\qquad s.t. D_{KL}(p_i(\tau)||\pi_\theta(\tau))\leq\epsilon$</li>*
    *<li>$\qquad$ ``S-step``: $\pi_\theta=argmin_\theta\sum_i D_{KL}(p_i(\tau)||\pi_\theta(\tau))$</li>*
</ol>
 
``C-step`` finds a new local policy that minimizes the cost and is close to the previously learnt global policy. ``S-step`` projects the local policy onto the constraint set $\Pi_\Theta$. The improvement of global policy is monotonic. Note that
\begin{equation}
D_{KL}(p_i(\tau)||\pi_\theta(r))=\sum^T_{t=1}\mathbb{E}_{p_i(x_t)}[D_{KL}(p_i(u_t|x_t)||\pi_\theta(u_t|x_t))]
\end{equation}

<h3>Nonlinear Global Policy and Unknown Dynamics</h3>
<li>Assuming dynamics is linear-Gaussian $p(x_{t+1}|x_t, u_t)=\mathcal{N}(f_{x_t} x_t + f_{u_t}u_t+f_{c_t}, F_t)$ and can be fitted w.r.t. the samples.</li> 
<li>Local time-varying policy can approximate global policy and can be fitted via linear regression or differentiating the policy.</li>
<li>Samples collected for dynamics fitting can also be used to train global policy.

*Algorithm 3*
<ol>
    *<li>for iteration k in 1:K do</li>*
    *<li>Generate samples $D_i=\{\tau_{i,j}\}$ by running either $p_i$ or $\pi_{\theta_i}$</li>*
    *<li>Fit linear-Gaussian dynamics $p_i(x_{t+1}|x_t, u_t)$ using samples in $D_i$</li>*
    *<li>Fit linearized global policy $\bar{\pi}_{\theta_i}(u_t|x_t)$ using samples in $D_i$</li>*
    *<li>$\qquad$ ``C-step``: $p_i=argmin_{p_i}\mathbb{E}_{p_i(\tau)}[\sum^T_{t=1}l(x_t, u_t)]\qquad s.t. D_{KL}(p_i(\tau)||\bar{\pi}_\theta(\tau))\leq\epsilon$</li>*
    *<li>$\qquad$ ``S-step``: $\pi_\theta=argmin_\theta\sum_i D_{KL}(p_i(\tau)||\pi_\theta(\tau))$</li>*
</ol>
 
<h3>Nonlinear Analysis</h3>

**Lemma 1** Let $\epsilon_t=max_{x_t} D_{KL}(p(\mu_t|x_t)||\pi_\theta(u_t|x_t)$. Then $D_{TV}(p(x_t)||\pi_\theta(x_t))\leq 2\sum^T_{t=1}\sqrt{2\epsilon_t}$ where $D_{TV}(p||q)=\frac{||p-q||_1}{2}$

Lemma 1 means that a bounded KL-divergence implies a boundded total divergence. 

**Lemma 2** If $D_{TV}(p(x_t)||\pi_\theta(x_t))\leq 2\sum^T_{t=1}\sqrt{2\epsilon_t}$, then the total cost of $\pi_\theta$ can be bounded as
\begin{equation}
\sum^T_{t=1}\mathbb{E}_{\pi_theta(x_t, u_t)}[\mathcal{l}(x_t, u_t)]\leq \sum^T_{t=1}[\mathbb{E}_{p(x_t, u_t)}[\mathcal{l}(x_t, u_t)]+\sqrt{2\epsilon_t}\underset{x_t, u_t}{max}\ \mathcal{l}(x_t, u_t)+2\sqrt{2\epsilon_t}Q_{max,t}]
\end{equation}
<center> where $Q_{max,t}=\sum^T_{t'=t}max_{x_{t'}, u_{t'}}\mathcal{l}(x_{t'},u_{t'})$, the maximum total cost from time t to T.</center>

Lemma 2 means that if local policy is updated to decrease its total cost, or decrease its KL-divergence against global policy, then the cost of global policy will also be decreased. **Cost of global policy, which is the left side of the equation, is majorized by the slacked cost of local policy, which is the right side of the equation.**

If $\epsilon$ is well chosen, the gap between the costs of global and local policies can be mitigated.
<li>Too large $\epsilon$ can produce unstable local policies.</li>
<li>Too small $\epsilon$ will prevent the local policies from improving significantlhy between iterations.</li>


